In [9]:
import speech_recognition as sr
import nltk

r = sr.Recognizer()

with sr.Microphone() as source:
    # read the audio data from the default microphone
    audio_data = r.record(source, duration=5)
    print("Recognizing...")
    # convert speech to text
    text = r.recognize_google(audio_data)
    print(text)
    
    tokens = nltk.word_tokenize(text)
    tagged = nltk.pos_tag(tokens)
    entities = nltk.chunk.ne_chunk(tagged)
    print(entities)
    

Recognizing...
Sam took Matthew's wrestling figure and broke it then he cried
(S
  (PERSON Sam/NNP)
  took/VBD
  (PERSON Matthew/NNP)
  's/POS
  wrestling/NN
  figure/NN
  and/CC
  broke/VBD
  it/PRP
  then/RB
  he/PRP
  cried/VBD)
